In [1]:
# ========================================
# CNN-Only Solar Power Forecasting
# ========================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
import matplotlib.pyplot as plt
import joblib

# ============================
# 1. Load dataset
# ============================
df = pd.read_csv(r"C:\Users\User\Desktop\solar_code\Task\plant1\Plant1_Merged_Final.csv")

# Ensure datetime format
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])

# Time features
df['hour'] = df['DATE_TIME'].dt.hour
df['minute'] = df['DATE_TIME'].dt.minute
df['time_fraction'] = df['hour'] + df['minute']/60

# Cyclical encoding
df['sin_time'] = np.sin(2 * np.pi * df['time_fraction'] / 24)
df['cos_time'] = np.cos(2 * np.pi * df['time_fraction'] / 24)

# Drop unused columns
df = df.drop(columns=['hour','minute','time_fraction','DATE_TIME'])

# Features and target
X = df.drop(columns=['AC_POWER'])
y = df['AC_POWER'].values   # raw watts

# ============================
# 2. Create sliding windows
# ============================
sequence_length = 6
X_seq, y_seq = [], []

for i in range(len(X) - sequence_length):
    X_seq.append(X.iloc[i:i+sequence_length].values)
    y_seq.append(y[i+sequence_length])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# ============================
# 3. Scale ONLY X
# ============================
scaler_X = MinMaxScaler()
X_flat = X_seq.reshape(-1, X_seq.shape[-1])
X_scaled = scaler_X.fit_transform(X_flat).reshape(X_seq.shape)

# y stays raw
y_raw = y_seq.reshape(-1,1)

# ============================
# 4. Train/Test split
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_raw, test_size=0.2, random_state=42, shuffle=True
)

# ============================
# 5. CNN Model (Only CNN)
# ============================
timesteps = X_scaled.shape[1]
n_features = X_scaled.shape[2]

inputs = Input(shape=(timesteps, n_features))
x = layers.Conv1D(64, 3, activation='relu', padding="causal")(inputs)
x = layers.Conv1D(32, 3, activation='relu', padding="causal")(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1)(x)

cnn_model = Model(inputs, outputs)
cnn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
cnn_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 6, 5)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 6, 64)          │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 6, 32)          │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,377 (36.63 KB)

 Trainable params: 9,377 (36.63 KB)

 Non-trainable params: 0 (0.00 B)

In [2]:
# ============================
# 6. Train CNN
# ============================
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, restore_best_weights=True)

history = cnn_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

# ============================
# 7. Evaluate
# ============================
y_pred = cnn_model.predict(X_test)

Epoch 1/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 117231784.0000 - mae: 6733.2139 - val_loss: 94294144.0000 - val_mae: 6290.6523
Epoch 2/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 58840868.0000 - mae: 5900.2524 - val_loss: 30277526.0000 - val_mae: 4871.6270
Epoch 3/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 20960618.0000 - mae: 3829.0149 - val_loss: 16392832.0000 - val_mae: 3042.6228
Epoch 4/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 12781813.0000 - mae: 2526.1748 - val_loss: 12447077.0000 - val_mae: 2268.3545
Epoch 5/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 10140468.0000 - mae: 2056.4268 - val_loss: 10143941.0000 - val_mae: 1955.8978
Epoch 6/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8612921.0000 - mae: 1805.6316 - val_loss: 8826906.0000 - val_mae: 1725.8558
Epoch 7/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 7756548.0000 - mae: 1620.6602 - val_loss: 7864844.5000 - val_mae: 1585.0139
Epoch 8/100
131/131 ━━━━━━━━━━

In [3]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"📊 CNN-only MSE: {mse:.2f}")
print(f"📊 CNN-only RMSE: {rmse:.2f}")
print(f"📊 CNN-only R²: {r2:.4f}")
print(f"📊 CNN-only MAE: {mae:.2f}")

📊 CNN-only MSE: 6030939.05
📊 CNN-only RMSE: 2455.80
📊 CNN-only R²: 0.9155
📊 CNN-only MAE: 1318.31


In [6]:








# ============================
# 10. Save CNN Model
# ============================
cnn_model.save(r"C:\Users\User\Desktop\solar_code\Task\plant1\cnn_model\model.h5", include_optimizer=False)
joblib.dump(scaler_X, r"C:\Users\User\Desktop\solar_code\Task\plant1\cnn_model\scaler_X.pkl")
print("✅ CNN-only model and scaler saved!")



✅ CNN-only model and scaler saved!


In [7]:
# ============================
# 1. Inference function
# ============================
def preprocess_input_for_inference(df_input, sequence_length=6):
    """
    Preprocess the input data for inference.
    df_input: DataFrame with columns [DATE_TIME, AMBIENT_TEMPERATURE, MODULE_TEMPERATURE, IRRADIATION]
    sequence_length: The number of past time steps (e.g., 6 time steps for 90 minutes)
    returns: Scaled input data ready for prediction
    """
    # Ensure datetime format
    df_input['DATE_TIME'] = pd.to_datetime(df_input['DATE_TIME'])

    # Time features
    df_input['hour'] = df_input['DATE_TIME'].dt.hour
    df_input['minute'] = df_input['DATE_TIME'].dt.minute
    df_input['time_fraction'] = df_input['hour'] + df_input['minute'] / 60.0

    # Cyclical encoding
    df_input['sin_time'] = np.sin(2 * np.pi * df_input['time_fraction'] / 24)
    df_input['cos_time'] = np.cos(2 * np.pi * df_input['time_fraction'] / 24)

    # Drop unused columns
    df_input = df_input.drop(columns=['hour', 'minute', 'time_fraction', 'DATE_TIME'])

    # Create sliding window
    X_seq = []
    for i in range(len(df_input) - sequence_length + 1):  # Fixed window creation
        X_seq.append(df_input.iloc[i:i+sequence_length].values)

    X_seq = np.array(X_seq)

    # Scale the input data
    X_flat = X_seq.reshape(-1, X_seq.shape[-1])
    X_scaled = scaler_X.transform(X_flat).reshape(X_seq.shape)

    return X_scaled

# ============================
# 2. Prediction function
# ============================
def predict_ac_power_for_inference(new_data):
    """
    Predict the AC power for the next 15 minutes using the trained CNN model.
    new_data: DataFrame with the latest time series data for prediction
    returns: predicted AC power
    """
    # Preprocess the input data
    X_scaled = preprocess_input_for_inference(new_data)

    # Predict using CNN model
    y_pred = cnn_model.predict(X_scaled)

    return y_pred

# ============================
# 3. Example Usage
# ============================
# Provide at least 6 consecutive rows (for sliding window)

sample_data = pd.DataFrame({
    "DATE_TIME": [
        "2020-05-15 06:00:00",
        "2020-05-15 06:15:00",
        "2020-05-15 06:30:00",
        "2020-05-15 06:45:00",
        "2020-05-15 07:00:00",
        "2020-05-15 07:15:00",
    ],
    "AMBIENT_TEMPERATURE": [24.08, 24.01, 23.97, 24.21, 24.53, 24.82],
    "MODULE_TEMPERATURE": [22.20, 22.35, 22.89, 24.44, 27.19, 28.88],
    "IRRADIATION": [0.0058, 0.0223, 0.0494, 0.0954, 0.1419, 0.1547]
})

# Make prediction for the next 15 min of AC_POWER
preds = predict_ac_power_for_inference(sample_data)
print("🔮 Predicted AC Power (Next 15 min):", preds)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
🔮 Predicted AC Power (Next 15 min): [[5763.205]]
